When game starts, all states are zero. State system python equivalent = functions for each class. set up class templates in case more than one battle is wanted.
player class
enemy class
animation can be a generic glitch
quantum can be imported later
typical battle:
intro screen where all states are initialized. if start button pressed, hide the screen and initiate runtime
a) roll die to see who goes first b) 

In [1]:
import math
import sys
import os
sys.path.insert(0, os.path.abspath('./..'))
from jupylet.sprite import Sprite
from jupylet.label import Label
from jupylet.app import App

In [4]:
app = App()

In [5]:
from qiskit import *
from math import pi

moves are quantum gates
- OBJECTIVE: PLAYER GETS HIGHEST PROBABILITY OF GETTING TO THE 1 STATE
- moves manipulate sphere
- negative phases are defenses
- record moves and apply gates to a circuit that moves the arrow of a bloch sphere
- at the end of the game, measure and see who wins
- extra feature: playback 
- make room to add more cards
- cards with turn cooldown
Setting up game:
- set up enemy and player moves (are cards that are given prechosen or are they chosen randomly from a predecided array)
- text in top corner showing current vector state of qubit (backend result)
- game start: define
- function player's move:
    - 1) waits for card user input.
    - 2) runs a sub-function that loads up cardUI and waits for user response. Depending on which one they click, it returns the selected value
    - 3) put the selected value in an array
    - 4) fade out card that was selected
    - 4) (port in cards)
- how will winning states be declared?
    - 1) if you get |1> you win

In [8]:
playerCards = []
enemyCards = []
winner = "nobody"


In [ ]:
label = Label('hello, world', color='cyan', font_size=32, x=10, y=10)

In [ ]:
def playerTurn():
    